In [76]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils
import torchvision.models as models

In [77]:
writer = SummaryWriter(log_dir='./data/实例1', filename_suffix="tb")
alexnet = models.alexnet(pretrained=True)  # 预训练好的Alexnet模型

path_img = "test_picture.png"
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]

norm_transform = transforms.Normalize(normMean, normStd)
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    norm_transform
])

In [78]:
img_pil = Image.open(path_img).convert('RGB')
img_tensor = img_transforms(img_pil)
img_tensor.unsqueeze_(0)  # CHW --> BCHW

convlayer1 = alexnet.features[0]  # AlexNet的第一个卷积层
fmap_1 = convlayer1(img_tensor)  # 64个卷积对应的特征图
fmap_1.shape

torch.Size([1, 64, 55, 55])

In [79]:
fmap_1.transpose_(0, 1)  # BCHW=(1, 64, 55, 55) --> (64, 1, 55, 55)
fmap_1_grid = vutils.make_grid(fmap_1, normalize=True, scale_each=True, nrow=8)

writer.add_image('feature map in conv1', fmap_1_grid, global_step=0, dataformats='CHW')  # 保存特征图
writer.close()

